In [ ]:
# Converts a multiline string of spike timestamps into a dictionary of neurons and their timestamps.
# Each neuron section starts with "neuron X:" followed by its spike timestamps on new lines.

def convert_timestamps_to_dict(text_block, neuron_prefix="neuron"):
    """
    Converts a multiline text block of spike timestamps into a dictionary keyed by neuron names.

    Parameters
    ----------
    text_block : str
        Multiline string where each neuron's block starts with a line like 'neuron X:' 
        followed by spike timestamps on the following lines.

    neuron_prefix : str, optional
        Prefix string used to detect neuron header lines (default = 'neuron').

    Returns
    -------
    neuron_data : dict
        Dictionary mapping each neuron name to a list of float timestamps. For example:
        neuron 1: [1654170515.71814, 1654170516.03764, 1654170516.07374]
        neuron 2: [1654170520.123456, 1654170520.523456]
    """
    lines = text_block.strip().splitlines()
    neuron_data = {}
    current_neuron = None
    current_timestamps = []

    for line in lines:
        line = line.strip()
        if not line:
            continue  # Skip blank lines
        if line.startswith(neuron_prefix):
            if current_neuron and current_timestamps:
                neuron_data[current_neuron] = current_timestamps
            current_neuron = line.rstrip(':')
            current_timestamps = []
        else:
            try:
                current_timestamps.append(float(line))
            except ValueError:
                raise ValueError(f"Invalid timestamp: {line}")

    if current_neuron and current_timestamps:
        neuron_data[current_neuron] = current_timestamps

    return neuron_data

# === Example usage ===

example_text = """
neuron 1:
1654170515.718140
1654170516.037640
1654170516.073740
neuron 2:
1654170520.123456
1654170520.523456
"""

neuron_data = convert_timestamps_to_dict(example_text)
print("Parsed neuron data:")
for neuron, timestamps in neuron_data.items():
    print(f"{neuron}: {timestamps}")


In [ ]:
# Processes neuron spike timestamps into normalized firing rates,
# calculates dot products over sliding time windows,
# and saves both firing rates and dot product matrices to Excel.

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def find_earliest_timestamp(neuron_data):
    """
    Finds the earliest timestamp across all neurons.

    Parameters
    ----------
    neuron_data : dict
        Dictionary mapping neuron names to lists of timestamps (float).

    Returns
    -------
    float
        Earliest timestamp in the dataset.
    """
    earliest = float('inf')
    for timestamps in neuron_data.values():
        if timestamps:
            earliest = min(earliest, timestamps[0])
    return earliest

def adjust_timestamps_and_calculate_rates(neuron_data, bin_size, common_start, total_duration=600):
    """
    Adjusts spike timestamps to a common time frame and bins firing rates.

    Parameters
    ----------
    neuron_data : dict
        Dictionary mapping each neuron to list of spike timestamps.
    bin_size : float
        Size of each time bin in seconds (e.g., 10).
    common_start : float
        Global start time for normalization (e.g., min timestamp across all neurons).
    total_duration : float
        Total duration in seconds (default 600s).

    Returns
    -------
    adjusted_data : dict
        Dictionary of adjusted spike timestamps per neuron.
    firing_rates : dict
        Dictionary of firing rates per bin per neuron.
    """
    adjusted_data = {}
    firing_rates = {}
    end_time = common_start + total_duration
    bins = np.arange(0, total_duration + bin_size, bin_size)

    for neuron, timestamps in neuron_data.items():
        adjusted = [t - common_start for t in timestamps if common_start <= t <= end_time]
        spike_counts, _ = np.histogram(adjusted, bins=bins)
        rates = spike_counts / bin_size
        adjusted_data[neuron] = adjusted
        firing_rates[neuron] = rates

    return adjusted_data, firing_rates

def save_firing_rates_to_excel(firing_rates, filename='actualdata_firing_rates.xlsx'):
    """
    Saves firing rates per neuron to an Excel file.

    Parameters
    ----------
    firing_rates : dict
        Dictionary of neuron-wise firing rates (list of floats).
    filename : str
        Excel filename to save output.
    """
    df = pd.DataFrame.from_dict(firing_rates, orient='index').transpose()
    df.insert(0, 'Interval', range(len(df)))
    df.to_excel(filename, index=False)

def calculate_and_save_dot_products(firing_rates, window_size=3, filename='actualdata_dot_products_normalized.xlsx'):
    """
    Calculates dot products between neuron firing rates using a sliding window approach,
    min-max normalizes the upper triangle, and saves each window as a sheet in an Excel file.

    Parameters
    ----------
    firing_rates : dict
        Dictionary of neuron firing rates (list of floats).
    window_size : int
        Number of bins per sliding window (default = 3).
    filename : str
        Excel file to save dot product matrices per window.

    Notes
    -----
    The upper triangle of each dot product matrix is min-max normalized per window.
    This transformation preserves the rank ordering of dot product values while 
    mapping them to the [0, 1] interval, which improves interpretability and 
    prevents extreme firing rate magnitudes from dominating similarity estimates.
    
    Min-max normalization is preferred here because:
    - It maintains the pairwise relationships between neurons within each time window
    - It avoids assumptions about distributional shape (unlike z-scoring)
    - It enables consistent thresholding or comparisons across animals or conditions
    """
    neuron_names = list(firing_rates.keys())
    num_neurons = len(neuron_names)
    num_intervals = len(next(iter(firing_rates.values())))
    num_windows = num_intervals - window_size + 1

    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        for start in range(num_windows):
            matrix = np.zeros((num_neurons, num_neurons))
            for i in range(num_neurons):
                for j in range(i + 1, num_neurons):
                    vec1 = firing_rates[neuron_names[i]][start:start + window_size]
                    vec2 = firing_rates[neuron_names[j]][start:start + window_size]
                    dot = np.dot(vec1, vec2)
                    matrix[i, j] = dot

            # Normalize upper triangle
            upper_triangle = np.triu_indices(num_neurons, k=1)
            scaler = MinMaxScaler()
            norm_values = scaler.fit_transform(matrix[upper_triangle].reshape(-1, 1)).flatten()
            matrix[upper_triangle] = norm_values

            # Save matrix as DataFrame
            df = pd.DataFrame(matrix, index=neuron_names, columns=neuron_names)
            df.to_excel(writer, sheet_name=f'Window {start}')

# === Example Usage ===

# Assuming neuron_data is a dictionary: { 'neuron 1': [timestamp1, timestamp2, ...], ... }
# For example:
# neuron_data = {
#     'neuron 1': [1654170515.718140, 1654170516.037640, ...],
#     'neuron 2': [1654170520.123456, 1654170521.001234, ...]
# }

bin_size = 10
duration = 600

common_start_time = find_earliest_timestamp(neuron_data)
adjusted_data, firing_rates = adjust_timestamps_and_calculate_rates(neuron_data, bin_size, common_start_time, total_duration=duration)

save_firing_rates_to_excel(firing_rates, filename='actualdata_firing_rates.xlsx')
calculate_and_save_dot_products(firing_rates, window_size=3, filename='actualdata_dot_products_normalized.xlsx')


In [ ]:
# Calculates the mean edge weight per neuron pair across all dot product matrices
# stored in an Excel file (one sheet per time window), and saves results to Excel.

import numpy as np
import pandas as pd
from itertools import combinations

def compute_mean_edge_weights(dot_product_excel_path, output_path='mean_edge_weights.xlsx'):
    """
    Computes the mean edge weight for each neuron pair by averaging
    dot products across all time windows.

    Parameters
    ----------
    dot_product_excel_path : str
        Path to the Excel file where each sheet represents a dot product matrix 
        for one time window. The matrix should have neurons as both row and column indices.

    output_path : str, optional
        Path to save the resulting mean edge weights as an Excel file. Default = 'mean_edge_weights.xlsx'

    Returns
    -------
    mean_edge_weights_df : pd.DataFrame
        DataFrame with columns: ['Neuron Pair', 'Mean Edge Weight']

    Notes
    -----
    This function assumes that each sheet contains a symmetric matrix with neuron names 
    as both row and column labels. Only the upper triangle (excluding the diagonal) is used.
    """
    xls = pd.ExcelFile(dot_product_excel_path)
    edge_weights = {}

    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name, index_col=0)

        for i, j in combinations(df.index, 2):  # Upper triangle only
            if (i, j) not in edge_weights:
                edge_weights[(i, j)] = []
            edge_weights[(i, j)].append(df.at[i, j])

    # Compute mean for each pair
    mean_edge_weights = {
        pair: np.mean(values) for pair, values in edge_weights.items()
    }

    mean_edge_weights_df = pd.DataFrame(
        list(mean_edge_weights.items()),
        columns=['Neuron Pair', 'Mean Edge Weight']
    )

    # Save to Excel
    mean_edge_weights_df.to_excel(output_path, index=False)
    return mean_edge_weights_df

# === Run the function directly ===

dot_product_file = "actualdata_dot_products_normalized.xlsx"   # Path to input file
output_file = "actualdata_mean_edge_weights.xlsx"              # Path to output file

mean_edge_weights_df = compute_mean_edge_weights(dot_product_file, output_file)
print("First 5 rows of mean edge weights:")
print(mean_edge_weights_df.head())


In [ ]:
# Calculates ISIs, shuffles ISIs to generate null spike trains for all neurons,
# plots ISI and spike trains for one example neuron, and saves the full set.

import numpy as np
import matplotlib.pyplot as plt

def calculate_isi(timestamps):
    """
    Computes interspike intervals (ISI) from a list of spike timestamps.

    Parameters
    ----------
    timestamps : list of float
        Spike timestamps for a single neuron.

    Returns
    -------
    np.ndarray
        Array of ISIs.
    """
    return np.diff(sorted(timestamps))

def shuffle_isi_and_create_spike_trains(isi, num_shuffles=1000):
    """
    Shuffles ISIs and creates null spike trains from cumulative sums.

    Parameters
    ----------
    isi : array-like
        Original ISIs to shuffle.
    num_shuffles : int
        Number of shuffled spike trains to generate.

    Returns
    -------
    list of np.ndarray
        List of shuffled spike trains.
    """
    spike_trains = []
    for _ in range(num_shuffles):
        shuffled = np.random.permutation(isi)
        spike_train = np.cumsum(shuffled)
        spike_trains.append(spike_train)
    return spike_trains

def plot_isi_and_spike_trains(original_timestamps, shuffled_spike_trains, filename='isi_and_spike_train_plots.png'):
    """
    Plots original and shuffled ISI distributions and spike trains.

    Parameters
    ----------
    original_timestamps : list of float
        Original spike timestamps.
    shuffled_spike_trains : list of np.ndarray
        Shuffled spike trains.
    filename : str
        Output image filename.
    """
    fig, axs = plt.subplots(5, 1, figsize=(15, 25))

    # Plot original ISI
    original_isi = calculate_isi(original_timestamps)
    axs[0].hist(original_isi, bins=50, color='blue', alpha=0.7)
    axs[0].set_title('Original ISI Distribution')
    axs[0].set_xlabel('Interspike Interval (s)')
    axs[0].set_ylabel('Frequency')

    # Plot shuffled ISI
    shuffled_isi = calculate_isi(shuffled_spike_trains[0])
    axs[1].hist(shuffled_isi, bins=50, color='red', alpha=0.7)
    axs[1].set_title('Shuffled ISI Distribution (1st Shuffle)')
    axs[1].set_xlabel('Interspike Interval (s)')
    axs[1].set_ylabel('Frequency')

    # Raster plots
    axs[2].eventplot(original_timestamps, color='blue')
    axs[2].set_title('Original Spike Train')
    axs[2].set_xlabel('Time (s)')

    for i in range(2):
        axs[3+i].eventplot(shuffled_spike_trains[i], color='red')
        axs[3+i].set_title(f'Shuffled Spike Train {i+1}')
        axs[3+i].set_xlabel('Time (s)')

    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

def save_shuffled_spike_trains(spike_trains_dict, filename='shuffled_spike_trains.npy'):
    """
    Saves shuffled spike trains dictionary to disk.

    Parameters
    ----------
    spike_trains_dict : dict
        Dictionary of neuron name → list of shuffled spike trains.
    filename : str
        Path to .npy file for saving.
    """
    np.save(filename, spike_trains_dict)

def load_shuffled_spike_trains(filename='shuffled_spike_trains.npy'):
    """
    Loads shuffled spike trains from a saved .npy file.

    Parameters
    ----------
    filename : str
        Path to the .npy file.

    Returns
    -------
    dict
        Dictionary of neuron name → list of shuffled spike trains.
    """
    return np.load(filename, allow_pickle=True).item()

# === Example Usage for Full Dataset ===

# Replace this with your full neuron dataset (10+ neurons)
neuron_data = {
    'neuron 1': [1654170515.7, 1654170516.0, 1654170517.0, 1654170519.0],
    'neuron 2': [1654170515.8, 1654170516.4, 1654170518.2, 1654170520.1],
    'neuron 3': [1654170515.9, 1654170516.2, 1654170516.5, 1654170516.9],
    # Add as many neurons as needed
}

# Shuffle ISIs for all neurons
shuffled_spike_trains = {
    neuron: shuffle_isi_and_create_spike_trains(calculate_isi(timestamps), num_shuffles=1000)
    for neuron, timestamps in neuron_data.items()
}

# Visualize one neuron (e.g., neuron 2) as example
plot_isi_and_spike_trains(
    original_timestamps=neuron_data['neuron 2'],
    shuffled_spike_trains=shuffled_spike_trains['neuron 2'],
    filename='isi_and_spike_train_plots.png'
)

# Save all shuffled trains to disk
save_shuffled_spike_trains(shuffled_spike_trains, filename='shuffled_spike_trains.npy')

# Verify saved structure
loaded = load_shuffled_spike_trains('shuffled_spike_trains.npy')
print(f"Loaded neurons: {list(loaded.keys())}")
print(f"Shuffles for neuron 2: {len(loaded['neuron 2'])}")
print(f"First 5 spikes of first shuffle: {loaded['neuron 2'][0][:5]}")


In [ ]:
# This script evaluates the significance of functional connectivity edge weights
# by comparing actual edge weights to a null distribution built from shuffled ISI spike trains.

import numpy as np
import pandas as pd
from scipy.stats import percentileofscore
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# === STEP 1: Firing Rate Binning ===
def calculate_firing_rates(timestamps, bin_size, total_duration):
    """
    Converts spike timestamps into binned firing rates.

    Parameters
    ----------
    timestamps : list of float
        Spike times for a neuron.
    bin_size : int
        Bin width in seconds.
    total_duration : int
        Total session duration in seconds.

    Returns
    -------
    rates : np.ndarray
        Firing rate per bin.
    """
    bins = np.arange(0, total_duration + bin_size, bin_size)
    spike_counts, _ = np.histogram(timestamps, bins=bins)
    return spike_counts / bin_size

# === STEP 2: Dot Product Over Sliding Windows ===
def calculate_dot_products_from_rates(firing_rates, window_size=3):
    """
    Computes dot product matrices for each sliding window of firing rates.

    Parameters
    ----------
    firing_rates : dict
        Neuron → firing rate vector.
    window_size : int
        Number of bins per dot product window.

    Returns
    -------
    list of pd.DataFrame
        List of dot product matrices (upper triangle normalized).
    """
    neurons = sorted(firing_rates.keys(), key=lambda x: int(x.split(' ')[1]))
    num_windows = len(firing_rates[neurons[0]]) - window_size + 1
    dot_product_matrices = []

    for start in range(num_windows):
        matrix = pd.DataFrame(0.0, index=neurons, columns=neurons)
        for i, n1 in enumerate(neurons):
            for j, n2 in enumerate(neurons):
                if i < j:
                    vec1 = firing_rates[n1][start:start + window_size]
                    vec2 = firing_rates[n2][start:start + window_size]
                    dot = np.dot(vec1, vec2)
                    matrix.at[n1, n2] = dot

        # Normalize the upper triangle using MinMaxScaler
        upper = np.triu_indices(len(neurons), k=1)
        matrix.values[upper] = MinMaxScaler().fit_transform(matrix.values[upper].reshape(-1, 1)).flatten()
        dot_product_matrices.append(matrix)

    return dot_product_matrices

# === STEP 3: Build Null Distribution ===
def build_edge_weight_distribution(shuffled_spike_trains, bin_size=10, duration=600, window_size=3):
    """
    Aggregates mean edge weights for all neuron pairs across shuffled spike trains.

    Parameters
    ----------
    shuffled_spike_trains : dict
        Neuron → list of shuffled spike trains.
    bin_size : int
        Width of each bin in seconds.
    duration : int
        Total session length.
    window_size : int
        Sliding window size for dot products.

    Returns
    -------
    edge_weight_distribution : list of float
        List of mean edge weights across all shuffled networks.
    """
    edge_weight_distribution = []
    neurons = sorted(shuffled_spike_trains.keys(), key=lambda x: int(x.split(' ')[1]))

    for shuffle_idx in range(len(shuffled_spike_trains[neurons[0]])):
        # Compute firing rates for this shuffle
        rates = {
            neuron: calculate_firing_rates(shuffled_spike_trains[neuron][shuffle_idx], bin_size, duration)
            for neuron in neurons
        }

        # Compute sliding-window dot products
        dp_matrices = calculate_dot_products_from_rates(rates, window_size)

        # Compute mean edge weights across all windows
        mean_weights = {}
        for matrix in dp_matrices:
            for i in range(len(neurons)):
                for j in range(i + 1, len(neurons)):
                    pair = (neurons[i], neurons[j])
                    mean_weights.setdefault(pair, []).append(matrix.iat[i, j])

        # Average the mean edge weights for each neuron pair and add to null distribution
        for pair, values in mean_weights.items():
            edge_weight_distribution.append(np.mean(values))

        # Optionally save dot product matrices from first 3 shuffles
        if shuffle_idx < 3:
            with pd.ExcelWriter(f'shuffled_dot_products_{shuffle_idx}.xlsx') as writer:
                for idx, mat in enumerate(dp_matrices):
                    mat.to_excel(writer, sheet_name=f'Window {idx}')

    return edge_weight_distribution

# === STEP 4: Significance Testing ===
def test_edge_significance(actual_edge_path, null_distribution, output_path='significance_results_with_p_values.xlsx'):
    """
    Compares actual edge weights to a null distribution and computes significance.

    Parameters
    ----------
    actual_edge_path : str
        Excel file with real edge weights (columns: 'Neuron Pair', 'Mean Edge Weight').
    null_distribution : list of float
        List of edge weights from null model.
    output_path : str
        Excel path to save results.

    Returns
    -------
    results_df : pd.DataFrame
        DataFrame with edge significance and p-values.
    """
    actual_df = pd.read_excel(actual_edge_path)
    actual_edges = dict(actual_df[['Neuron Pair', 'Mean Edge Weight']].values)

    significance = {}
    p_values = {}

    for pair, actual in actual_edges.items():
        percentile = percentileofscore(null_distribution, actual)
        p = min(percentile, 100 - percentile) / 100
        p_values[pair] = p
        significance[pair] = 'TRUE' if p < 0.05 else 'FALSE'

    results = pd.DataFrame({
        'Neuron Pair': list(significance.keys()),
        'P-Value': list(p_values.values()),
        'Significance': list(significance.values())
    })

    def format_p(p):
        if p < 0.001:
            return f'***{p:.4f}'
        elif p < 0.01:
            return f'**{p:.4f}'
        elif p < 0.05:
            return f'*{p:.4f}'
        else:
            return f'{p:.4f}'

    results['P-Value with Asterisks'] = results['P-Value'].apply(format_p)
    results.to_excel(output_path, index=False)
    return results

# === STEP 5: Null Distribution Plot ===
def plot_edge_weight_distribution(null_distribution, bins=50):
    """
    Plots histogram of null edge weight distribution.

    Parameters
    ----------
    null_distribution : list of float
        Edge weights from shuffled spike trains.
    bins : int
        Number of histogram bins.
    """
    plt.hist(null_distribution, bins=bins, alpha=0.75)
    plt.title('Edge Weight Null Distribution')
    plt.xlabel('Mean Edge Weight')
    plt.ylabel('Frequency')
    plt.show()

# === FULL PIPELINE RUN ===

# Load shuffled spike trains
shuffled_spike_trains = np.load("shuffled_spike_trains.npy", allow_pickle=True).item()

# Build null distribution of edge weights from all shuffles
null_dist = build_edge_weight_distribution(
    shuffled_spike_trains,
    bin_size=10,
    duration=600,
    window_size=3
)

# Plot null distribution
plot_edge_weight_distribution(null_dist)

# Test real edge weights against null
results_df = test_edge_significance(
    actual_edge_path='actualdata_mean_edge_weights.xlsx',
    null_distribution=null_dist,
    output_path='significance_results_with_p_values.xlsx'
)

# Display first few significant edges
print("Significance testing results (top 5 rows):")
print(results_df.head())
